In [22]:
!python -m pip install --upgrade pip
%pip install wget

Note: you may need to restart the kernel to use updated packages.


In [9]:
!python -m wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


Saved under input.txt


In [10]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("dataset length: ", len(text))
#print(text[:1000])

dataset length:  1115394


In [13]:
# Get the unique set of characters in corpus, sorted
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [14]:
# tokenize corpus 
# convert input to some sequence of integers
# translate char - int
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

# encoder: convert each char to int index
# decoder: take int and translate to string 
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l]) 

print(encode("hi there"))
print(decode(encode("hi there")))




[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [ ]:
#SentencePiece from Google is another encode/decode that uses sub-word units
#instead of individual characters 
%pip install tiktoken

import tiktoken
enc = tiktoken.get_encoding("gpt-2")
assert enc.decode(enc.encode("hello world")) == "hello world"

In [19]:
#pytorch
%pip install torch 

     -------------------------------------- 162.6/162.6 MB 7.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\carde\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [23]:
# spearate train and validation data sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]



In [24]:
#split corpus into blocks for efficiency
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [26]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

# these are the 8 examples spelled out from the training set
# the 8-size is for efficiency AND context
# as little as 1 character context allows a predicition
# more than block size input will be superflous

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [31]:
torch.manual_seed(1337)
batch_size = 4 # parallel independent sequences 
block_size = 8 # context length

def get_batch(split):
    # generate small batch of data inputs x and target y
    data = train_data if split == 'train' else val_data
    # random offsets in the training data
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)
print("-----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when input is [24] the target is: 43
when input is [24, 43] the target is: 58
when input is [24, 43, 58] the target is: 5
when input is [24, 43, 58, 5] the target is: 57
when input is [24, 43, 58, 5, 57] the target is: 1
when input is [24, 43, 58, 5, 57, 1] the target is: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39
when input is [44] the target is: 53
when input is [44, 53] the target is: 56
when input is [44, 53, 56] the target is: 1
when input is [44, 53, 56, 1] the target is: 58
when input is [44, 53, 56, 1, 58]

In [37]:
#bigram language model
import torch
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # every int in input correspond to row in embed table
        # (Batch, Time, Channel) - logits - score of next character 
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            #rearrange torch to fit loss function
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # loss function - negative log likelihood = cross entropy
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        #idx is (B, T) array of indices in the current context
        # generate should take (B, T) to (B, T+1)
        for _ in range(max_new_tokens):
            # get prediction
            logits, loss = self(idx)
            # filter to only last time step (bigram)
            logits = logits[:, -1, :] # format is (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # format is (B, C)
            # generate next sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # shape is (B, 1)
            # concatenate to input stream
            idx = torch.cat((idx, idx_next), dim=1) # shape is (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# 65 possible characters
# current loss = 4.8
# expected: -ln(1/65) = 4.17 so our loss is a little high

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [38]:
# batch is 1, time is 1. 1x1 tensor
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

#garbage output because of untrained model
# bigram model has entire input stream but uses only last character
# potential improvement is to use entire input


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [39]:
# train the model
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)# high learning rate due to small training data


In [46]:
batch_size = 32
for steps in range(10000):
    #samples a batch of data
    xb, yb = get_batch('train')
    #evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.450434684753418


In [53]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))
# model is improving


S:
S m mo,
And topeator kn s tte! ad sthasshore moinsou'e he II S:
AROUClly chorallly ICARiteand y hichivemmeveie elll?
mee me hele pes b:
GHit cen, foug setwio ousaro buntasd tl thit ome totho-fotend blourve aser thair fellale t s iee l penket tt he. t re hallive ho Lon Wive w he
ITENontovod
BOLORI hyourn LAUT: nt menr wes ndese.
Wh menoou nthathar to makeinoy bus ou kealy te berd ayo t th s n.

Thilamand.
SAhe ber f beawon t st thanie, winord n thad
Myorerom tls

Y:

CHIZANC:
I hebe otloun, gu
